# 4IF Data Mining Lab
Nadine Saadalla, Noémie Varjabedian, Eléonore Dravet  
February 2025


# Introduction
During this Data Mining project, we will be analyzing the geolocation data of photographs taken in the Grand Lyon. The data comes from the Flickr database and was taken between 2010 and 2019. We will explore different clustering methods to identify areas of interest for tourism in the metropolis, i.e. areas where many photographs are taken. Finally, we'll work on photograph captions to assign descriptions to our areas of interest

# Python set-up

In this section, we will set up a virtual python environment and install all necessary packages to run the code.

In [ ]:
! source ./vitualenvpython/bin/activate
! which python3


In [ ]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
#maping library
! pip install folium
# association rules
! pip install mlxtend==0.23.3

! pip install nbformat==5.9.2 

In [ ]:
# load pandas to deal with the data
import pandas as pd
# plotting
import matplotlib.pyplot as plt
import folium
import numpy as np

# Data loading and cleaning

We start by viewing the current dataset.

In [ ]:
data = pd.read_table("flickr_data2.csv", sep=",")
data.head()

### Cleaning data in unnamed columns

In [ ]:
data.info()

We notice the three lines    

16  Unnamed: 16          142 non-null     float64  
17  Unnamed: 17          0 non-null       float64  
18  Unnamed: 18          2 non-null       float64  

which indicate that some lines have values in unnamed columns, probably due to a wrong use of the separators ','. They represent a very small ration of the overall data (~150/420 000) so we chose to delete those lines without further investigation. 

In [ ]:
unnamed_columns = data.columns[data.columns.str.contains('^Unnamed')]
data = data.loc[~data["Unnamed: 16"].notna(),:]
data = data.loc[~data["Unnamed: 17"].notna(),:]
data = data.loc[~data["Unnamed: 18"].notna(),:]
data.info()

The lines have been deleted ("0 non-null"). The columns still appear, we can delete them. 

In [ ]:
data=data.drop(columns=unnamed_columns)
data.info()

### Spaces

We notice that except for 'id', the column names start with a space. To manipulate the columns more easily, we remove these spaces. 

In [ ]:
data=data.rename(columns=lambda x: x.strip())

### Incoherent values
We can look at the statistics to check that they are coherent.

In [ ]:
data.describe()

We see that the max date_taken_year is 2238, which is impossible. This pushes us to delete all rows where the date taken is more than 2025 (the current year), and print all the years where pictures have been taken in our dataset, to check that everything is coherent.


In [ ]:
index=data[data['date_taken_year']>2025].index
data.drop(index=index,axis=1)
taken_years = data["date_taken_year"].unique()
uploded_years = data["date_upload_year"].unique()
print(f" When pictures are taken : {taken_years}\n")
print(f" When pictures are uploaded : {uploded_years}")

<a id="missing-vals"></a>
### Missing Values

In [ ]:
data.isna().sum()

We see that the missing values are either from the tags, the title or the upload_year. For our analysis on the coordinates, these information are not determining and we can choose to keep the corresponding data. However, in order to run our algorithm, we sample the data (our CPUs are not powerful enough to run with all the data) and keep only a thousand points, so we may as well keep the data with tags for our text analysis later.

In [ ]:
data=data.drop(index=data[data['tags'].isna()].index)

<a id="duplicates"></a>
### Duplicates

For our analysis to be pertinent, we do not want to count several times pictures taken by the same person, in the same place, in the same hour. This allows us to count visits. This avoids getting influenced by someone taking burst photos as well. Hence we remove the duplicates using the following columns. 

In [ ]:
columns_for_duplicates=['user', 'lat', 'long','date_taken_hour', 'date_taken_day', 'date_taken_month','date_taken_year', 'date_upload_minute', 'date_upload_hour','date_upload_day', 'date_upload_month', 'date_upload_year']
data=data.drop_duplicates(subset=columns_for_duplicates)

We now have ~90 00 points from the ~420 000 initial. We still have enough values. 

In [ ]:
data.info()

We can visualize all the points on a map as a last check of our values. We choose 1000 random tagged points.

In [ ]:
m = folium.Map([45.762611,4.832805	], zoom_start=14)
data_sample = data.sample(1000)

for index, row in data_sample.iterrows():
    folium.Marker(
        location=[row["lat"],row["long"]],
        icon=folium.DivIcon(html=f"""<svg width="20px" height="20px" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
        <path d="M12 9.5C13.3807 9.5 14.5 10.6193 14.5 12C14.5 13.3807 13.3807 14.5 12 14.5C10.6193 14.5 9.5 13.3807 9.5 12C9.5 10.6193 10.6193 9.5 12 9.5Z" fill="#e63946"/>
        </svg>""")
    ).add_to(m)

m

Everything looks cleaned. We can save our cleaned data to a new csv and work with this version as for now. We will not have do do the treatment again since we can just directly use the data_cleaned.csv. 

In [ ]:
data.to_csv('./data_cleaned.csv',index=False)

# Analyzing clsutering methods

In [ ]:

data_cleaned=pd.read_csv('./data_cleaned.csv')
data_sample = data_cleaned.sample(1000, random_state=42)
cluster_colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#FFA500', '#800080']

## k-means Clustering and Find the Optimal Number of Clusters using Elbow Method

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull, QhullError
import folium

Before using the Elbow method, we observe the different clusters we obtain by changing the value of k. 

### Elbow method 
We use the elbow method to determine the best parameter k for our K-Means clustering.

In [ ]:
inertias = []
for i in range(1,30):
  kmeans=KMeans(n_clusters=i,init='k-means++')
  kmeans.fit(data_sample[["lat", "long"]])
  inertias.append(kmeans.inertia_)

plt.plot(range(1,30),inertias)
plt.show()

The optimal value seems to be k=7. When we visualize the result on the map, there are not enough clusters to represent touristic places. We hence chose to use a higher k to adapt to the reality.

P.S. : We only show clusters than contain more than 10 points. Samller clusters are considered irrelevant. This changes nothing for k=7 but for a bigger k or for other methods, this rule has an impact on the printed map.

### k-means = 7 | 100 maps

In [ ]:
k = 7

kmeans = KMeans(n_clusters=k, init='k-means++')
kmeans.fit(data_sample[["lat", "long"]])

m = folium.Map(location=[45.762611, 4.832805], zoom_start=14)

labels = kmeans.labels_

clustered_points = data_sample.assign(cluster=labels)

for cluster_id in clustered_points['cluster'].unique():
    cluster_points = clustered_points[clustered_points['cluster'] == cluster_id]
    points = cluster_points[['lat', 'long']].values

    if len(points) >= 10:  # We only show clusters with more than 10 points.
        try:
            hull = ConvexHull(points)
            hull_points = points[hull.vertices]  
            
            
            folium.Polygon(
                locations=hull_points,
                color=cluster_colors[cluster_id % len(cluster_colors)],
                weight=2,
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except QhullError:
            print(f"Cluster {cluster_id}: QhullError occurred; skipping hull computation.")
    else:
        print(f"Cluster {cluster_id}: Less than 10 points; skipping perimeter.")

for _, row in data_sample.iterrows():
    folium.Marker(
        location=[row["lat"], row["long"]],
        icon=folium.DivIcon(html=f"""
        <svg width="10px" height="10px" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
            <circle cx="12" cy="12" r="5" fill="black"/>
        </svg>
        """)
    ).add_to(m)

m.save("cluster_map_k_means_7.html")


In [ ]:
k = 100

kmeans = KMeans(n_clusters=k, init='k-means++')
kmeans.fit(data_sample[["lat", "long"]])

m = folium.Map(location=[45.762611, 4.832805], zoom_start=14)

labels = kmeans.labels_

clustered_points = data_sample.assign(cluster=labels)

for cluster_id in clustered_points['cluster'].unique():
    cluster_points = clustered_points[clustered_points['cluster'] == cluster_id]
    points = cluster_points[['lat', 'long']].values

    if len(points) >= 10:  # We only show clusters with more than 10 points.
        try:
            hull = ConvexHull(points)
            hull_points = points[hull.vertices]  
            
            
            folium.Polygon(
                locations=hull_points,
                color=cluster_colors[cluster_id % len(cluster_colors)],
                weight=2,
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except QhullError:
            print(f"Cluster {cluster_id}: QhullError occurred; skipping hull computation.")
    else:
        print(f"Cluster {cluster_id}: Less than 10 points; skipping perimeter.")

for _, row in data_sample.iterrows():
    folium.Marker(
        location=[row["lat"], row["long"]],
        icon=folium.DivIcon(html=f"""
        <svg width="10px" height="10px" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
            <circle cx="12" cy="12" r="5" fill="black"/>
        </svg>
        """)
    ).add_to(m)

m.save("cluster_map_k_means_100.html")


## Task #5: Cluster Evaluation using Silhouette Coefficient | A FAIRE

To evaluate the quality of the clustering, we can use **Silhouette Coefficient**. The Silhouette Coefficient for a sample is given by $(b - a) / max(a, b)$ where `b` is the distance between a sample and the nearest cluster that the sample is not a part of, and `a` is the mean intra-cluster distance (i.e. the mean distance between a sample and all other samples in the same cluster). 

The silhouette score ranges from -1 to 1 and indicates how well each data point fits within its assigned cluster:

* Score near +1 means:
    - The data point is far from neighboring clusters
    - The point is well-matched to its cluster
    - Indicates very distinct, well-separated clustering
* Score near 0 means:
    - The data point is close to the decision boundary between clusters
    - The point could potentially belong to either cluster
    - Suggests overlapping or not well-defined clusters
* Score near -1 means:
    - The data point might be assigned to the wrong cluster
    - The point is closer to points in another cluster than its own
    - Indicates poor clustering or potential misassignments

We can use [`sklearn.metrics.silhouette_score`](https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.silhouette_score.html) and [`sklearn.metrics.silhouette_samples`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_samples.html)

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples

**QUESTIONS**

* For k-means clustering with `k=3`, calculate Silhouette score for each data point, for each cluster and average silhouette score 
* Display Silhouette score plot
* Comment

In [ ]:
kmeans=KMeans(n_clusters=3,init='k-means++')
for_silhouette_df = scaled_data_df.copy()
for_silhouette_df["labels"] = labels

for_silhouette_df = for_silhouette_df.sort_values(by=["labels"])

for_silhouette_df["silhouettes"] = silhouette_samples(for_silhouette_df[features],for_silhouette_df["labels"])

cluster_silhouette_score = for_silhouette_df.groupby(["labels"]).mean()["silhouettes"]

average_silhouette_score = for_silhouette_df["silhouettes"].mean()

In [ ]:
import matplotlib.cm as cm

In [ ]:
fig = plt.figure(figsize = (20,20))
fig, (ax1) = plt.subplots(1, 1)
y_lower = 10
for i in range(3):
    ith_cluster_silhouette_values = for_silhouette_df.loc[for_silhouette_df["labels"] == i]

    ith_cluster_silhouette_values.sort_values(by=["silhouettes"])

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / 3)
    ax1.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        ith_cluster_silhouette_values,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )

    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    y_lower = y_upper + 10

ax1.set_title("The silhouette plot for the various clusters.")
ax1.set_xlabel("The silhouette coefficient values")
ax1.set_ylabel("Cluster label")

ax1.axvline(x=average_silhouette_score, color="red", linestyle="--")

ax1.set_yticks([]) 
ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

In [ ]:
# ANSWER


As general guidelines, the plot can be interpreted by looking at:
* *the thickness of the clusters (number of points)*;
* check if any cluster has many negative values;
* check the consistency of the silhouette widths within clusters;
* the average value. Recall that in general, the following interpretation applies:
    - \> 0.7: Strong clustering structure
    - 0.5 - 0.7: Reasonable clustering structure
    - 0.25 - 0.5: Weak clustering structure
    - < 0.25: No substantial clustering structure


* Cluster Silhouette scores. 

YOUR COMMENT: TO COMPLETE

## Hierarchical Clustering : single, complete and ward

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In this part, we analyse the 3 variations of hierarchical clustering.

In [ ]:
clustering_single = AgglomerativeClustering(n_clusters=100, linkage='single').fit(data_sample[["lat", "long"]] )

In [ ]:
clustering_complete = AgglomerativeClustering(n_clusters=100, linkage='complete').fit(data_sample[["lat", "long"]] )

In [ ]:
clustering_ward = AgglomerativeClustering(n_clusters=100, linkage='ward').fit(data_sample[["lat", "long"]] )

After observing on the map the different results, we determine that the best hierarchical cluserting method is the ward one. 

In [ ]:
print(clustering_ward.labels_)

In [ ]:
from collections import Counter


m = folium.Map(location=[45.762611, 4.832805], zoom_start=14)

labels = clustering_ward.labels_

clustered_points = data_sample.assign(cluster=labels)


cluster_names = {}

for cluster_id in clustered_points['cluster'].unique():
    if cluster_id < 2:
        print(f"Skipping cluster {cluster_id} (outlier)")
        continue

    cluster_points = clustered_points[clustered_points['cluster'] == cluster_id]

    excluded_tags = {"lyon", "france", "europe", "francia", "rhône", "auvergnerhônealpes", "fra", "iphoneography", "rhôneaples", "frankreich", "frankrig"}  # Tags to ignore
    all_tags = []
    for tags in cluster_points['tags'].dropna():  # Drop empty tag entries
        filtered_tags = [tag for tag in tags.split(',') if tag.lower().strip() not in excluded_tags]
        all_tags.extend(filtered_tags)

    # Determine the most common tag(s)
    if all_tags:
        tag_counts = Counter(all_tags)
        most_common_tags = [tag for tag, count in tag_counts.most_common(3)]  # Top 3 tags
        cluster_name = ', '.join(most_common_tags)  # Combine for naming
    else:
        cluster_name = f"Cluster {cluster_id}"  # Default name if no tags available

    # Store the name for this cluster
    cluster_names[cluster_id] = cluster_name
    print(f"Cluster {cluster_id} named: {cluster_name}")


    points = cluster_points[['lat', 'long']].values

    if len(points) >= 10: 
        try:
            hull = ConvexHull(points)
            hull_points = points[hull.vertices]
            
            folium.Polygon(
                locations=hull_points,
                color=cluster_colors[cluster_id % len(cluster_colors)], 
                weight=2,
                fill=True,
                fill_opacity=0.2,
                tooltip=cluster_names[cluster_id]
            ).add_to(m)
        except QhullError:
            print(f"Cluster {cluster_id}: QhullError occurred; skipping hull computation.")
    else:
        print(f"Cluster {cluster_id}: Less than 3 points; skipping perimeter.")

for _, row in data_sample.iterrows():
    folium.Marker(
        location=[row["lat"], row["long"]],
        icon=folium.DivIcon(html=f"""
        <svg width="10px" height="10px" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
            <circle cx="12" cy="12" r="5" fill="black"/>
        </svg>
        """),
        tooltip=row["tags"]  # Show tags on hover
    ).add_to(m)

m.save("cluster_map_clustering_ward.html")
m

In [ ]:

import requests

# Flickr API details
FLICKR_API_KEY = "c9f9183c6f9f862b589a12aa27b9c8e6"
FLICKR_SECRET = "f46c69ef000f0202"

# Function to get the photo's server-id and secret from Flickr API
def get_flickr_image_url(photo_id):
    url = "https://api.flickr.com/services/rest/"
    params = {
        "method": "flickr.photos.getInfo",
        "api_key": FLICKR_API_KEY,
        "photo_id": photo_id,
        "format": "json",
        "nojsoncallback": 1
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if "photo" in data:
        photo = data["photo"]
        server_id = photo["server"]
        secret = photo["secret"]
        
        # Construct image URL
        image_url = f"https://live.staticflickr.com/{server_id}/{photo_id}_{secret}.jpg"
        return image_url
    
    return None  # Return None if photo details are unavailable

# Create Folium map
m = folium.Map(location=[45.762611, 4.832805], zoom_start=14)

data_sample2 = data_cleaned.sample(100, random_state=42)

# Add markers with images as tooltips
for _, row in data_sample2.iterrows():
    image_url = get_flickr_image_url(row["id"])
    
    if image_url:
        tooltip_html = f'<img src="{image_url}" width="200px" height="150px"/>'
    else:
        tooltip_html = "Image not available"

    folium.Marker(
        location=[row["lat"], row["long"]],
        icon=folium.DivIcon(html=f"""
        <svg width="10px" height="10px" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
            <circle cx="12" cy="12" r="5" fill="black"/>
        </svg>
        """),
        tooltip=tooltip_html
    ).add_to(m)

# Save and display map
m.save("cluster_map_with_images.html")
m


In [ ]:
! pip install ipyleaflet

In [ ]:
from ipyleaflet import Map, Marker, Popup
from IPython.display import display
import requests
import json

# Your Flickr API Key
FLICKR_API_KEY = "c9f9183c6f9f862b589a12aa27b9c8e6"

# Function to fetch the image URL from Flickr API
def get_flickr_image_url(photo_id):
    flickr_api_url = f"https://api.flickr.com/services/rest/?method=flickr.photos.getInfo&api_key={FLICKR_API_KEY}&photo_id={photo_id}&format=json&nojsoncallback=1"
    
    response = requests.get(flickr_api_url)
    data = response.json()
    
    if "photo" in data:
        photo = data["photo"]
        server_id = photo["server"]
        secret = photo["secret"]
        
        # Construct and return the correct image URL
        return f"https://live.staticflickr.com/{server_id}/{photo_id}_{secret}.jpg"
    
    return None  # Return None if the image is unavailable

# Create the map centered on Lyon
m = Map(center=(45.762611, 4.832805), zoom=14)

# Add markers for each data point
for _, row in data_sample.iterrows():
    lat, lon, photo_id = row["lat"], row["long"], row["id"]
    
    # Create a marker
    marker = Marker(location=(lat, lon))

    # Define the hover event
    def on_hover(event, photo_id=photo_id, marker=marker):
        image_url = get_flickr_image_url(photo_id)  # Fetch image dynamically

        if image_url:
            popup_content = f'<img src="{image_url}" width="250px" height="180px"/>'
        else:
            popup_content = "Image not available"

        popup = Popup(location=marker.location, child=display(popup_content), close_button=False, auto_close=True)
        m.add_layer(popup)

    # Attach the hover event listener
    marker.on_interaction(on_hover)
    
    # Add marker to the map
    m.add_layer(marker)

# Display the map
display(m)


## Task 7: Apply DBSCAN

**QUESTIONS**

* Apply [sklearn.cluster.DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) algorithm
* Identify the best values for `eps` and `min_sanples` by varying the values within a range and by using Silhouette coefficient
* Apply DBSCAN with the best parameters found
* Print number of clusters and noise points

In [24]:
# DBSCAN
from sklearn.cluster import DBSCAN

We try with random eps and min_samples and evaluate the result with the Silhouette coefficient. After adjusting, we find a satisfying score of 0.698 for eps = 0.03 and min_samples=5.

In [56]:
# ANSWER
data_sample = data_sample.sample(1000, random_state=42)

clustering_db = DBSCAN(eps=0.03, min_samples=5).fit(data_sample[["lat", "long"]] )
DBlabels=clustering_db.labels_

silhouette_score(data_sample[["lat", "long"]], DBlabels)

0.6976672605751058

In [57]:
from scipy.spatial import ConvexHull, QhullError

# Initialize the map
m = folium.Map(location=[45.762611, 4.832805], zoom_start=12)

# Assign cluster labels
labels = DBlabels

# Group points by cluster
clustered_points = data_sample.assign(cluster=labels)

# Define colors for clusters (extendable)
cluster_colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#FFA500', '#800080']

# Track if any polygons are added
polygon_added = False

# Loop through each cluster and compute its perimeter
for cluster_id in clustered_points['cluster'].unique():

    if cluster_id < 0:
        print(f"Skipping cluster {cluster_id} (outlier)")
        continue

    cluster_points = clustered_points[clustered_points['cluster'] == cluster_id]
    points = cluster_points[['lat', 'long']].values  # Extract points as (lat, long)

    if len(points) >= 3:  # ConvexHull requires at least 3 points
        try:
            hull = ConvexHull(points)
            hull_points = points[hull.vertices]  # Get the hull points
            
            # Add the perimeter as a polygon to the map
            folium.Polygon(
                locations=hull_points,
                color=cluster_colors[cluster_id % len(cluster_colors)],  # Use modulo for repeating colors
                weight=2,
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
            polygon_added = True  # Mark that a polygon was successfully added
        except QhullError:
            print(f"Cluster {cluster_id}: QhullError occurred; skipping hull computation.")
    else:
        print(f"Cluster {cluster_id}: Less than 3 points; skipping perimeter.")

# Add markers for all points
for _, row in data_sample.iterrows():
    folium.Marker(
        location=[row["lat"], row["long"]],
        icon=folium.DivIcon(html=f"""
        <svg width="10px" height="10px" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
            <circle cx="12" cy="12" r="5" fill="black"/>
        </svg>
        """)
    ).add_to(m)

# Save and display the map
m.save("cluster_map_dbscan.html")
m

Skipping cluster -1 (outlier)


When we visualize the clusters, we realize that one cluster is way too big. We can try other parameters that, even if they give a lower Silhouette score, might correspond better to the reality of touristic places of interest. We take a smaller eps to break down the big cluster in smaller ones. We can see that other clusters disappear (or have than 3 points, since we have chosen to only consider clusters with more than three photos taken). We then reduce min_samples to try to have more clusters elsewhere.
The result is not as satisfying as other methods of clustering.

In [61]:
# ANSWER
data_sample = data_sample.sample(1000, random_state=42)

clustering_db = DBSCAN(eps=0.001, min_samples=3).fit(data_sample[["lat", "long"]])
DBlabels=clustering_db.labels_

silhouette_score(data_sample[["lat", "long"]], DBlabels)

0.055993961992064606

In [62]:
from scipy.spatial import ConvexHull, QhullError

# Initialize the map
m = folium.Map(location=[45.762611, 4.832805], zoom_start=12)

# Assign cluster labels
labels = DBlabels

# Group points by cluster
clustered_points = data_sample.assign(cluster=labels)

# Define colors for clusters (extendable)
cluster_colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#FFA500', '#800080']

# Track if any polygons are added
polygon_added = False

# Loop through each cluster and compute its perimeter
for cluster_id in clustered_points['cluster'].unique():

    if cluster_id < 0:
        print(f"Skipping cluster {cluster_id} (outlier)")
        continue

    cluster_points = clustered_points[clustered_points['cluster'] == cluster_id]
    points = cluster_points[['lat', 'long']].values  # Extract points as (lat, long)

    if len(points) >= 3:  # ConvexHull requires at least 3 points
        try:
            hull = ConvexHull(points)
            hull_points = points[hull.vertices]  # Get the hull points
            
            # Add the perimeter as a polygon to the map
            folium.Polygon(
                locations=hull_points,
                color=cluster_colors[cluster_id % len(cluster_colors)],  # Use modulo for repeating colors
                weight=2,
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
            polygon_added = True  # Mark that a polygon was successfully added
        except QhullError:
            print(f"Cluster {cluster_id}: QhullError occurred; skipping hull computation.")
    else:
        print(f"Cluster {cluster_id}: Less than 3 points; skipping perimeter.")

# Add markers for all points
for _, row in data_sample.iterrows():
    folium.Marker(
        location=[row["lat"], row["long"]],
        icon=folium.DivIcon(html=f"""
        <svg width="10px" height="10px" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
            <circle cx="12" cy="12" r="5" fill="black"/>
        </svg>
        """)
    ).add_to(m)

# Save and display the map
m.save("cluster_map_dbscan.html")
m

Skipping cluster -1 (outlier)
Cluster 7: QhullError occurred; skipping hull computation.
Cluster 14: QhullError occurred; skipping hull computation.
Cluster 17: QhullError occurred; skipping hull computation.
Cluster 18: QhullError occurred; skipping hull computation.
Cluster 21: QhullError occurred; skipping hull computation.
Cluster 25: QhullError occurred; skipping hull computation.
Cluster 28: QhullError occurred; skipping hull computation.
Cluster 30: QhullError occurred; skipping hull computation.
Cluster 39: QhullError occurred; skipping hull computation.
Cluster 42: QhullError occurred; skipping hull computation.
Cluster 44: QhullError occurred; skipping hull computation.
Cluster 46: QhullError occurred; skipping hull computation.


## Task 8: Cluster Characterisation using Apriori algorithm

Now, we would like to describe the obtained cluster. To do so, let's use frequent pattern mining and in particular **Apriori algorithm**. 

**QUESTIONS**
* First, convert numerical features to categorical (low, medium, high) based on quantiles. Add binary columns, e.g. `sepal length low`, `sepal length medium`, `sepal length high` depending on the values

In [ ]:
# ANSWER


To find association, we are going to use [mlxtend.frequent_patterns.apriori](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/).

In [ ]:
# frequent patterns
from mlxtend.frequent_patterns import apriori

**QUESTIONS**

* Use apriori algorithm to find frequent patterns for each cluster
* Then among these itemsets, find those that are not frequent for other clusters

In [ ]:
# ANSWER